# Data enrichment

### <u>Content:<u>

[1) Load data ](#load_data) 

[2) Wochentag
    
[3) Ferien Anzahl Personen
    
[4) Koordinaten

[5 Wetter


In [1]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import seaborn as sns
import scipy
from scipy.spatial.distance import cdist
rcParams['figure.figsize']=10,8

#### 1) Load data

In [68]:
#read the reservation data 
df = pd.read_csv('reservation_data_2019-2021_incl_capacity/reservation_data_2019-2021_incl_capacity.csv', 
                 parse_dates=["date"], date_parser=lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S"))         
        
print('reservations: ')
display(df.tail(1))

# read the holiday data, include a date range column
df_schulferien = pd.read_csv('data_preprocessed/Schulferien.csv', dtype={"canton": "string", "population": "int32"})
df_schulferien['start'] = pd.to_datetime(df_schulferien['holidays_start'])
df_schulferien['end'] = pd.to_datetime(df_schulferien['holidays_end'])
df_schulferien = df_schulferien.drop(columns=["holidays_start", "holidays_end"])

print('schulferien: ')
display(df_schulferien.tail(1))

# train station coordinates data
df_coordinates = pd.read_csv('data_preprocessed/dienststellen.csv')
df_coordinates = df_coordinates[["abk_bahnhof", "lat", "lon"]]
df_coordinates = df_coordinates[df_coordinates['abk_bahnhof'].notna()]
display(df_coordinates.tail(1))

# weather data
df_weather = pd.read_csv("data_preprocessed/weather.csv")
display(df_weather.tail(1))

reservations: 


,Unnamed: 0,res_id,res_dt,date,train_nr,line,reserved,capacity,bp_from,bp_to,dep_ist,dep_soll,arr_ist,arr_soll,res_delta_ist,res_delta_soll,res_delta_valid
226735,226735,246187,31/10/2021 17:16,2021-10-31,1635,IC 51,1,6.0,BI,DMT,31/10/2021 19:49,31/10/2021 19:49,31/10/2021 20:17,31/10/2021 20:18,9216.0,9210,True


schulferien: 


,canton,population,start,end
161,national,7917100,2022-12-25,2022-12-26


C:\Users\tabea\miniconda3\envs\geo_env\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,abk_bahnhof,lat,lon
47113,BRIA,47.260083,7.942674


,station_id,validdate,leisure_biking:idx,snow_depth:cm,t_2m:C,precip_24h:mm,weather_symbol_1h:idx,effective_cloud_cover:octas
4745,station_id,validdate,leisure_biking:idx,snow_depth:cm,t_2m:C,precip_24h:mm,weather_symbol_1h:idx,effective_cloud_cover:octas


#### 2) Wochentag <a name="stat"></a>

- Add a feature for weekday: 'weekday' and 'month'

In [73]:
df['weekday'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month


#### 3) Ferien <a name="stat"></a>
- Add a feature for number of people in holiday canton: n_holiday

In [74]:
# for each date, get the number of people in Switzerland who 
# are either on school holiday or national holiday..
def get_holiday_people(date):
    print(date)
    filtered_holidays = df_schulferien[(df_schulferien['start']<=date)&(df_schulferien['end']>=date)]
    is_national_holiday = (filtered_holidays["canton"]=="national").sum()
    if is_national_holiday:
        people = 7917100
    elif not(filtered_holidays.empty):
        filtered_holidays = filtered_holidays[filtered_holidays["canton"]!="national"]
        people = sum(filtered_holidays["population"])
    else: people=0
    return people

In [77]:
# filter df, only 2021 data
df=df[df['date']>='2021-01-01']

#df['holiday_people'] = df.apply(lambda row : get_holiday_people(row['date']), axis = 1)


#### 3) Coordinates <a name="hr"></a>

In [28]:
# full join for start train station 
df = pd.merge(df, df_coordinates, left_on='bp_from', right_on='abk_bahnhof')
df = df.drop(columns=['abk_bahnhof']).rename(columns={"lat": "lat_from", "lon": "lon_from"})

# full join for destination
df = pd.merge(df, df_coordinates, left_on='bp_to', right_on='abk_bahnhof')
df = df.drop(columns=['abk_bahnhof']).rename(columns={"lat": "lat_to", "lon": "lon_to"})
df

,Unnamed: 0,res_id,res_dt,date,train_nr,line,reserved,capacity,bp_from,bp_to,...,dep_soll,arr_ist,arr_soll,res_delta_ist,res_delta_soll,res_delta_valid,lat_from,lon_from,lat_to,lon_to
0,0,0,29/03/2019 00:00,2019-04-01,510,IC 5,1,NaN,ZUE,NE,...,01/04/2019 07:03,01/04/2019 08:33,01/04/2019 08:32,284667.0,284634,False,47.378177,8.540212,46.996727,6.935702
1,283,290,06/04/2019 00:00,2019-04-06,1528,IC 5,1,NaN,ZUE,NE,...,06/04/2019 16:30,06/04/2019 18:02,06/04/2019 18:01,59422.0,59418,False,47.378177,8.540212,46.996727,6.935702
2,357,365,07/04/2019 00:00,2019-04-07,1524,IC 5,1,NaN,ZUE,NE,...,07/04/2019 14:30,07/04/2019 16:00,07/04/2019 16:01,52216.0,52218,False,47.378177,8.540212,46.996727,6.935702
3,611,619,11/04/2019 00:00,2019-04-11,1530,IC 5,1,NaN,ZUE,NE,...,11/04/2019 17:30,11/04/2019 19:00,11/04/2019 19:01,63021.0,63018,False,47.378177,8.540212,46.996727,6.935702
4,731,739,13/04/2019 00:00,2019-04-13,534,IC 5,1,NaN,ZUE,NE,...,13/04/2019 19:03,13/04/2019 20:31,13/04/2019 20:32,68741.0,68634,False,47.378177,8.540212,46.996727,6.935702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226731,226144,245085,29/10/2021 14:34,2021-10-29,1073,IC 6/61,1,12.0,SP,IO,...,29/10/2021 15:05,29/10/2021 15:30,29/10/2021 15:28,1880.0,1860,True,46.686396,7.680103,46.690500,7.869000
226732,226173,245116,29/10/2021 19:54,2021-10-29,1083,IC 6/61,1,9.0,SP,IO,...,29/10/2021 20:05,29/10/2021 20:26,29/10/2021 20:28,622.0,660,True,46.686396,7.680103,46.690500,7.869000
226733,181405,196032,19/08/2021 17:43,2021-08-20,807,IC 8,1,15.0,VI,FR,...,20/08/2021 05:55,20/08/2021 06:11,20/08/2021 06:11,43970.0,43968,True,46.294029,7.881465,46.588908,7.651418
226734,193313,208346,02/09/2021 19:20,2021-09-03,807,IC 8,1,9.0,VI,FR,...,03/09/2021 05:55,03/09/2021 06:10,03/09/2021 06:11,38104.0,38148,True,46.294029,7.881465,46.588908,7.651418


In [ ]:
#Create month column (Run this only once)
df_air['MONTH']=df_air.index.month     
df_air.reset_index(inplace=True)

- ##### Drop column NMHC_GT; it has 90% missing data

In [ ]:
df_air['CO_GT']=df_air['CO_GT'].fillna(df_air.groupby(['MONTH','HOUR'])['CO_GT'].transform('mean'))
df_air['NOX_GT']=df_air['NOX_GT'].fillna(df_air.groupby(['MONTH','HOUR'])['NOX_GT'].transform('mean'))
df_air['NO2_GT']=df_air['NO2_GT'].fillna(df_air.groupby(['MONTH','HOUR'])['NO2_GT'].transform('mean'))

- Print left missing values

In [ ]:
print('Left out missing value:')
df_air.isnull().sum()

In [ ]:
# Fill left out NaN values with hourly average value
df_air['CO_GT'] = df_air['CO_GT'].fillna(df_air.groupby(['HOUR'])['CO_GT'].transform('mean'))
df_air['NOX_GT'] = df_air['NOX_GT'].fillna(df_air.groupby(['HOUR'])['NOX_GT'].transform('mean'))
df_air['NO2_GT'] = df_air['NO2_GT'].fillna(df_air.groupby(['HOUR'])['NO2_GT'].transform('mean'))

#### 4) Understand correlation between variables<a name="corr"></a>

 - Use heatmap to see correlation between variables
 - Describe the heatmap using your own words

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_air.corr(), annot=True,fmt='.1f')
plt.show()

 - PT08_S3_NOX negatively correlated to other air pollutants
 - All the other pollutants strongly correlated
 - weather variables (T, RH, AH) have relatively weak correlation with air quality parameters
 - temperature strongly correlated with absolute- and relative humidity 
 - target variable RH: highest linear correlation to temperature and hour of the day

#### 5) Try to understand degree of linearity between RH output and other input features<a name="lin"></a>

 - plot all X-features against output variable RH using `sns.lmplot`
 - describe the results

In [ ]:
#plot all X-features against output variable RH

In [ ]:
# remove date and time columns 
df_air =df_air.drop(['DATE', 'TIME'], axis=1)

# melt dataframe and plot all variables against X-features
df_air_melt = pd.melt(df_air, 'RH', var_name='cols', value_name='vals')
g = sns.lmplot(x='vals', y='RH', col='cols', col_wrap=5, data=df_air_melt, sharex=False)

- Most features don't seem to correlate with the target variable (the plots are just point clouds).
- Temperature has a somewhat linear relationship with RH, but the residuals are not normally distributed around the linear regression line (for high temperatures, RH is always overestimated by the regressor).
- Hour and month are clearly related to RH, but the relationship is clearly not linear.

### 6) Linear Regression<a name="LR"></a>

In [ ]:
from sklearn.preprocessing import StandardScaler         #import normalisation package
from sklearn.model_selection import train_test_split      #import train test split
from sklearn.linear_model import LinearRegression         #import linear regression package
from sklearn.metrics import mean_squared_error,mean_absolute_error   #import mean squared error and mean absolute error

- ##### Define Feature (X) and Target (y)

In [ ]:
X = df_air.drop('RH', axis=1).values #X-input features
y = df_air.RH.values #y-input features           

- Plot distribution of target variable. Do we need to use stratified splitting of the data? Why?

I would argue that we don't need stratified splitting in this case, because the distribution of RH is symmetric. I would use stratified splitting in the case of a skewed distribution or in a classification problem, if there are few data points of a certain class. 

In [ ]:
plt.hist(y, bins=50)
plt.title('Distribution of RH values')
plt.show()

- ##### Train test split
split the data into train and test with test size and 30% and train size as 70%, use a random seed

In [ ]:
np.random.seed(42)

# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# for stratified splitting: used code from this website
# https://michaeljsanders.com/2017/03/24/stratify-continuous-variable.html
# bins=np.linspace(0, 100, 10)
# y_binned = np.digitize(y, bins)
# stratify=y_binned)

In [ ]:
print('Training data size:'), print(X_train.shape)
print('Test data size:'), print(X_test.shape)

- ##### Normalize data using `StandardScaler`

In [ ]:
# fit standard scaler on training data
scaler=StandardScaler().fit(X_train)

In [ ]:
# transform training and test data
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

 - ##### Train the Linear Regression model

In [ ]:
LinearModel=LinearRegression().fit(X_train_scaled, y_train)

- Print intercept and slope

In [ ]:
print('Intercept: ', LinearModel.intercept_)
print('Slope: ', LinearModel.coef_)

- Predict on the test data
- Compute and print performance metrics as RMSE. This will be our baseline

In [ ]:
y_pred = LinearModel.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE baseline', "%.3f" % rmse)

# plot distribution of predicted RH values
# plt.hist(y_pred, bins=50)
# plt.title('Distribution of predicted RH values')
# plt.show()

#### <u>6a) Conclusion of baseline linear regression model:<a name="LRcon"></a>
    
Write your conclusions here: 
    
A linear regressor is not the best choice for the given task. The lmplots show that the linear relationships of most features with the target variable are weak. Moreover the linear regressor predicts some negative values for RH, which does not make sense physically. A reasonable model should account for this restriction.

### 7) Feature engineering and testing model:<a name="FE"></a>

Try with multiple feature combination and see if RMSE is improving

- ##### write function to measure RMSE with different combinations of features (try at least 6 combinations of your choice)

(remember to comment the function!)

In [ ]:
def train_test_RMSE(df_air, feat_):
    
    '''
    The function train_test_RMSE returns the RMSE for different combinations 
    of features feat_ of the dataframe df_air'''
    
    # select feature and target variables 
    X = df_air[feat_].values #X-input features
    y = df_air.RH.values #y-input features  
    
    # split train and test data
    np.random.seed(21)
    
    X_trainR, X_testR, y_trainR, y_testR = train_test_split(X, y, test_size=0.3, random_state=21)
    
    # scale features
    scaler=StandardScaler().fit(X_trainR)
    X_trainR_scaled=scaler.transform(X_trainR)
    X_testR_scaled=scaler.transform(X_testR)
    
    # apply linear regression model
    LinearModel=LinearRegression().fit(X_trainR_scaled, y_trainR)
    y_predR=LinearModel.predict(X_testR_scaled)
    
    # return rmse value
    return np.sqrt(mean_squared_error(y_testR, y_predR))

In [ ]:
feat_1=['T']
feat_2=['T', 'AH']
feat_3=['T', 'AH', 'HOUR']
feat_4=['AH', 'T', 'MONTH', 'HOUR']
feat_5=list(df_air.drop(['RH', 'T', 'MONTH', 'HOUR'], axis=1).columns.tolist())
feat_6=list(df_air.drop(['RH', 'T'], axis=1).columns.tolist())
feature_combinations = [feat_1, feat_2, feat_3, feat_4, feat_5, feat_6]

In [ ]:
rmse_dict = {}
for feat_comb in feature_combinations:
    rmse_combination = train_test_RMSE(df_air, feat_comb)
    rmse_dict[rmse_combination] = feat_comb
    print('Feature combination: ', feat_comb)
    print('RMSE: ', "%.3f" % rmse_combination, '\n')

#### <u>7a) Conclusion of Feature Engineering and testing:<a name="FEcon"></a>
    
    - Write your conclusions here
    
    1) Using only a subset of features decreases the accuracy of the model. However, a combination of few features such as combination 2 (AH, T) results in a model that is almost as accurate with much reduced number of parameters. These two features are the most important ones to include in the model, which makes sense due to the direct physical relationship between T, RH and AH. 
    2) Using all features except for temperature (combination 6) results in a much worse model.
    3) Using temperature alone results in a much worse model than when combining it with AH.
    4) Using the temporal features (hour, month) does not result in a good model. Even though the two variables are related to the target variable, the non-linear relationship is not captured well by the Linear Regressor.


### 8) Decision Tree Regression<a name="DT"></a>

Let us try to apply Decision tree regression technique and see if any improvement happens

In [ ]:
from sklearn.tree import DecisionTreeRegressor         #Decision tree regression model
from sklearn.model_selection import cross_val_score    #import cross validation score package
from sklearn.model_selection import GridSearchCV        #import grid search cv

- ##### Fit the DT model and predict:

In [ ]:
DecisionTree=DecisionTreeRegressor().fit(X_train_scaled, y_train)
y_pred = DecisionTree.predict(X_test_scaled)

- ##### RMSE of RH prediction

In [ ]:
#calculate RMSE
print('RMSE of Decision Tree Regression:', "%.3f" % np.sqrt(mean_squared_error(y_test, y_pred)))

#### <u>Conclusion:<u>(Decision Tree Regression)

    - Write your conclusions here
    The decision tree works much better than the linear regression because it can account for non-linear relationships between the features and target variable.

### 9) Random Forest Regression<a name="RF"></a>

- apply Random Forest regression and measure RMSE

In [ ]:
from sklearn.ensemble import RandomForestRegressor           #import random forest regressor

- ##### Fit the RF model and predict

In [ ]:
RF=RandomForestRegressor().fit(X_train_scaled, y_train)
y_pred = RF.predict(X_test_scaled)

- ##### RMSE of RH prediction

In [ ]:
#Calculate RMSE
print('RMSE of predicted RH in RF model:', "%.3f" % np.sqrt(mean_squared_error(y_test, y_pred)))

--------

- ##### Try to improve on baseline RF model: use `GridSearchCV` to search between different hyperparameters and plot the resulting RMSE
    - use different numbers of estimators
    - use cv of 5 or 10
    - use the correct scoring function
    - then, use the best model hyperparameters to predict on the test data

In [ ]:
# i used the example here as a reference
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 400, num = 4)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [5, 10, 100]

# Create the random grid, for now only look for the best number of trees 
grid = {'n_estimators': n_estimators}
               #'max_features': max_features,
               #'max_depth': max_depth}

In [ ]:
grid_search = GridSearchCV(estimator = RF, param_grid = grid, cv=5)

# Fit the model with different parameters from the grid to the data
grid_search.fit(X_train_scaled, y_train)

#print best parameters
grid_search.best_params_


In [ ]:
y_pred = grid_search.best_estimator_.predict(X_test_scaled)

In [ ]:
print('RMSE using RF grid search method', "%.3f" % np.sqrt(mean_squared_error(y_test, y_pred)))  

- Write here your conclusions regarding the Grid Search method. Did the performance improve? How much?

The default number of trees in the RF model is 100. From GridSearch CV we find that increasing the number of trees to 200 or 400 improves the performance. The RMSE of the predicted target variable decreases by around 1-2% compared to the default RF model. Depending on the application this can be a significant improvement of performance but the additional computation effort may be also considered.

- ### Plot box plots of the error <a name="bxplot"></a>

    - how are the error distributed over different ranges of RH?
    - plot the box plots of absolute errors vs different output range) 

In [ ]:
df_error = pd.DataFrame(data={'RH': y_test, 'RH_predicted': y_pred, 'abs_error': np.abs(y_pred - y_test)})
df_error['RH_bins']=pd.cut(df_error['RH'], np.arange(0, 110, 10))
sns.boxplot(x=df_error.RH_bins, y=df_error.abs_error)
plt.show()

The largest errors occur at the edge of the distribution.
This could be caused by the distribution of the target variable (less training samples at the edges) and could possibly be improved by using stratified splitting.

#### <u>Conclusion: Random Forest
    
    - Write your conclusions here


The random Forest algorithm further increases the accuracy. It is less prone to overfitting compared to the decision tree model. 

### 10) Support Vector Machine<a name="SVM"></a>

- apply SVR and measure RMSE

In [ ]:
from sklearn.svm import SVR           #import support vector regressor

In [ ]:
SVM=SVR(kernel='rbf').fit(X_train_scaled, y_train)
y_pred = SVM.predict(X_test_scaled)

In [ ]:
#Calculate RMSE of SVR
print('RMSE of SVR model:', "%.3f" % np.sqrt(mean_squared_error(y_test, y_pred)))

## Conclusion:<a name="conclusion"></a>

 - Summarize here your conclusions regarding the models used 
 
 


In this assignment I learned to apply different regression models.
However, to solve the task in reality, I think it would make sense to consider the time series structure of the data. The simple linear regressor cannot learn much from the features 'month' and 'hour' even though they are clearly related to the target variable (in a non-linear way). The other regressors such as the decision tree and RF account for the non-linearity and therefore provide more accurate predictions of the target variable.

Considering the physical relationships between temperature, AH and RH could also improve the model, e.g. compute an additional feature for the saturation vapor pressure (function of temperature).